<a href="https://colab.research.google.com/github/aditeyabaral/gpt2-implementation/blob/main/openai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# OpenAI based Implementation

[Article](https://medium.com/@ngwaifoong92/beginners-guide-to-retrain-gpt-2-117m-to-generate-custom-text-content-8bb5363d8b7f)

[Notebook Guide](https://github.com/ak9250/gpt-2-colab/blob/master/GPT_2.ipynb)

# Installing and Setting up Environment

In [15]:
import os

In [16]:
!git clone https://github.com/nshepperd/gpt-2.git

Cloning into 'gpt-2'...
remote: Enumerating objects: 375, done.
remote: Total 375 (delta 0), reused 0 (delta 0), pack-reused 375
Receiving objects: 100% (375/375), 4.43 MiB | 6.31 MiB/s, done.
Resolving deltas: 100% (204/204), done.


In [17]:
os.chdir("gpt-2")
!pip3 install tensorflow==1.15.0
!pip install -r requirements.txt

In [18]:
!python3 download_model.py 345M
!export PYTHONIOENCODING=UTF-8

Fetching checkpoint: 1.00kit [00:00, 1.24Mit/s]                                                     
Fetching encoder.json: 1.04Mit [00:00, 72.5Mit/s]                                                   
Fetching hparams.json: 1.00kit [00:00, 1.44Mit/s]                                                   
Fetching model.ckpt.data-00000-of-00001: 1.42Git [00:19, 73.1Mit/s]                                 
Fetching model.ckpt.index: 11.0kit [00:00, 11.0Mit/s]                                               
Fetching model.ckpt.meta: 927kit [00:00, 72.3Mit/s]                                                 
Fetching vocab.bpe: 457kit [00:00, 66.8Mit/s]                                                       


In [19]:
os.chdir("src")

In [20]:
import json
import numpy as np
import tensorflow as tf
import model, sample, encoder
import pandas as pd
import numpy as np

# Loading Training Data

In [21]:
df = pd.read_csv("/content/Simpsons.csv")
df.dropna(inplace = True)
df.drop_duplicates(inplace = True)
df.reset_index(inplace = True)
df.drop(columns = "index", inplace = True)
print(df.shape)
df.head()

(126646, 2)


,raw_character_text,spoken_words
0,Miss Hoover,"No, actually, it was a little of both. Sometim..."
1,Lisa Simpson,Where's Mr. Bergstrom?
2,Miss Hoover,I don't know. Although I'd sure like to talk t...
3,Lisa Simpson,That life is worth living.
4,Edna Krabappel-Flanders,The polls will be open from now until the end ...


In [22]:
def character_slice(character):
  if character in np.unique(df[["raw_character_text"]]):
    return df[df["raw_character_text"] == character]

In [23]:
CHARACTER = "Lisa Simpson"

character_df = character_slice(CHARACTER)
print(character_df.shape)
character_df.head()

(10144, 2)


,raw_character_text,spoken_words
1,Lisa Simpson,Where's Mr. Bergstrom?
3,Lisa Simpson,That life is worth living.
8,Lisa Simpson,Mr. Bergstrom! Mr. Bergstrom!
10,Lisa Simpson,Do you know where I could find him?
12,Lisa Simpson,"The train, how like him... traditional, yet en..."


## Adding delimiters

This is optional - allows for retention of training data structure

In [24]:
train_data = "\n<|endoftext|>\n".join(character_df["spoken_words"].values)

In [25]:
os.chdir("..")
with open("corpus.txt","w") as f:
  f.write(train_data)

# Training GPT-2

This is an endless loop, terminate execution to stop process

In [26]:
!PYTHONPATH=src /content/gpt-2/train.py --dataset /content/gpt-2/src/corpus.txt --model_name '345M'


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.




2020-10-20 06:48:37.976891: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 AVX512F FMA
2020-10-20 06:48:37.980985: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2000140000 Hz
2020-10-20 06:48:37.981193: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x315b480 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2020-10-20 06:48:37.981222: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2020-10-20 06:48:3

# Generation

## User Prompt

In [27]:
!python3 src/interactive_conditional_samples.py --top_k 40 --model_name "345M"



2020-10-20 06:51:04.198317: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-10-20 06:51:04.235850: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-10-20 06:51:04.236412: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: Tesla T4 major: 7 minor: 5 memoryClockRate(GHz): 1.59
pciBusID: 0000:00:04.0
2020-10-20 06:51:04.236695: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.0
2020-10-20 06:51:04.237709: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10.0
2020-10-20 06:51:04.238717: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcufft.so.10.0

## Random Sampling

In [28]:
!python3 src/generate_unconditional_samples.py --model_name "345M" | tee /tmp/samples



2020-10-20 06:52:15.379814: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-10-20 06:52:15.411850: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-10-20 06:52:15.412416: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: Tesla T4 major: 7 minor: 5 memoryClockRate(GHz): 1.59
pciBusID: 0000:00:04.0
2020-10-20 06:52:15.412689: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.0
2020-10-20 06:52:15.413693: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10.0
2020-10-20 06:52:15.414698: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcufft.so.10.0